In [1]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/datasource.zip

--2023-01-14 17:51:39--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/datasource.zip
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4249 (4.1K) [application/zip]
Saving to: ‘datasource.zip’

datasource.zip      100%[===================>]   4.15K  --.-KB/s    in 0s      

2023-01-14 17:51:39 (383 MB/s) - ‘datasource.zip’ saved [4249/4249]



In [36]:
import pandas as pd
from datetime import datetime
import glob,os
import xml.etree.ElementTree as ET

#UNZIP THE FILE

In [6]:
!unzip datasource.zip -d dealership_data


Archive:  datasource.zip
  inflating: dealership_data/used_car_prices1.csv  
  inflating: dealership_data/used_car_prices2.csv  
  inflating: dealership_data/used_car_prices3.csv  
  inflating: dealership_data/used_car_prices1.json  
  inflating: dealership_data/used_car_prices2.json  
  inflating: dealership_data/used_car_prices3.json  
  inflating: dealership_data/used_car_prices1.xml  
  inflating: dealership_data/used_car_prices2.xml  
  inflating: dealership_data/used_car_prices3.xml  


#SET PATHS

In [7]:
tmpfile    = "temp.tmp"               # file used to store all extracted data
logfile    = "logfile.txt"            # all event logs will be stored in this file
targetfile = "transformed_data.csv"   # file where transformed data is stored

#CSV EXTRACT FUNCTION

In [8]:
def extract_from_csv(file_to_process):
    dataframe = pd.read_csv(file_to_process)
    return dataframe

#### JSON Extract Function

In [9]:
def extract_from_json(file_to_process):
    dataframe = pd.read_json(file_to_process,lines=True)
    return dataframe

# XML EXTRACT FUNCTION

In [47]:
def extract_from_xml(file_to_process):
  #https://www.analyticsvidhya.com/blog/2021/06/implementing-python-to-learn-data-engineering-etl-process/

    dataframe = pd.DataFrame(columns=['car_model','year_of_manufacture','price', 'fuel'])

    tree = ET.parse(file_to_process) 

    root = tree.getroot() 

    for person in root: 

        car_model = person.find("car_model").text 

        year_of_manufacture = int(person.find("year_of_manufacture").text)

        price = float(person.find("price").text) 

        fuel = person.find("fuel").text 

        dataframe = dataframe.append({"car_model":car_model, "year_of_manufacture":year_of_manufacture, "price":price, "fuel":fuel}, ignore_index=True) 

        return dataframe

In [39]:
def extract():
    extracted_data = pd.DataFrame(columns=['car_model','year_of_manufacture','price', 'fuel']) # create an empty data frame to hold extracted data
    
    #process all csv files
    for csvfile in glob.glob("dealership_data/*.csv"):
        extracted_data = extracted_data.append(extract_from_csv(csvfile), ignore_index=True)
        
    #process all json files
    for jsonfile in glob.glob("dealership_data/*.json"):
        extracted_data = extracted_data.append(extract_from_json(jsonfile), ignore_index=True)
    
    #process all xml files
    for xmlfile in glob.glob("dealership_data/*.xml"):
        extracted_data = extracted_data.append(extract_from_xml(xmlfile), ignore_index=True)
        
    return extracted_data

#TRANSFORM

In [40]:
def transform(data): 
        data['price'] = round(data.price,2)
        return data

#LOADING 

In [41]:
def load(targetfile,data_to_load):
    data_to_load.to_csv(targetfile)  

# LOGGING

In [42]:
def log(message):
    timestamp_format = '%Y-%h-%d-%H:%M:%S' # Year-Monthname-Day-Hour-Minute-Second
    now = datetime.now() # get current timestamp
    timestamp = now.strftime(timestamp_format)
    with open("logfile.txt","a") as f:
        f.write(timestamp + ',' + message + '\n')

#RUNNING ETL JOBS

In [43]:
log("ETL Job Started")

In [44]:
log("Extract phase Started")
extracted_data = extract()
log("Extract phase Ended")
extracted_data

,car_model,year_of_manufacture,price,fuel
0,alto k10,2016,4253.731343,Petrol
1,ignis,2017,7313.432836,Petrol
2,sx4,2011,6567.164179,Petrol
3,alto k10,2014,3731.343284,Petrol
4,wagon r,2013,4328.358209,Petrol
...,...,...,...,...
58,etios g,2014,6119.402985,Petrol
59,fortuner,2014,29835.820896,Diesel
60,corolla altis,2016,21985.074627,Diesel
61,corolla altis,2013,10373.134328,Petrol


In [45]:
log("Transform phase Started")
transformed_data = transform(extracted_data)
log("Transform phase Ended")
transformed_data 

,car_model,year_of_manufacture,price,fuel
0,alto k10,2016,4253.73,Petrol
1,ignis,2017,7313.43,Petrol
2,sx4,2011,6567.16,Petrol
3,alto k10,2014,3731.34,Petrol
4,wagon r,2013,4328.36,Petrol
...,...,...,...,...
58,etios g,2014,6119.40,Petrol
59,fortuner,2014,29835.82,Diesel
60,corolla altis,2016,21985.07,Diesel
61,corolla altis,2013,10373.13,Petrol


In [46]:
log("Load phase Started")
load(targetfile,transformed_data)
log("Load phase Ended")